In [71]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [179]:
import geopandas as gpd

shapefile = 'mapsource/ne_50m_admin_0_countries.shp'
datafile = 'dataset/globalterrorismdb_0718dist.csv'

gdf = gpd.read_file(shapefile)[['ADMIN', 'ADM0_A3', 'geometry']]
gdf.columns = ['country', 'country_code', 'geometry']
gdf.head(3)

,country,country_code,geometry
0,Zimbabwe,ZWE,"POLYGON ((31.28789 -22.40205, 31.19727 -22.344..."
1,Zambia,ZMB,"POLYGON ((30.39609 -15.64307, 30.25068 -15.643..."
2,Yemen,YEM,"MULTIPOLYGON (((53.08564 16.64839, 52.58145 16..."


In [180]:
import pandas as pd

datafile = 'dataset/globalterrorismdb_0718dist.csv'

df = pd.read_csv(datafile, encoding='iso-8859-1', usecols = ['iyear', 'imonth', 'iday', 'country', 'country_txt', 'region_txt', 'provstate', 'city', 
                            'success', 'suicide', 'attacktype1_txt', 'targtype1', 'targtype1_txt', 'longitude', 'latitude', 
                            'targsubtype1', 'targsubtype1_txt', 'natlty1', 'natlty1_txt', 'weaptype1', 'weaptype1_txt', 'gname'], low_memory=False)
df.head(3)

,iyear,imonth,iday,country,country_txt,region_txt,provstate,city,latitude,longitude,...,attacktype1_txt,targtype1,targtype1_txt,targsubtype1,targsubtype1_txt,natlty1,natlty1_txt,gname,weaptype1,weaptype1_txt
0,1970,7,2,58,Dominican Republic,Central America & Caribbean,NaN,Santo Domingo,18.456792,-69.951164,...,Assassination,14,Private Citizens & Property,68.0,Named Civilian,58.0,Dominican Republic,MANO-D,13,Unknown
1,1970,0,0,130,Mexico,North America,Federal,Mexico city,19.371887,-99.086624,...,Hostage Taking (Kidnapping),7,Government (Diplomatic),45.0,"Diplomatic Personnel (outside of embassy, cons...",21.0,Belgium,23rd of September Communist League,13,Unknown
2,1970,1,0,160,Philippines,Southeast Asia,Tarlac,Unknown,15.478598,120.599741,...,Assassination,10,Journalists & Media,54.0,Radio Journalist/Staff/Facility,217.0,United States,Unknown,13,Unknown


In [183]:
df_asia = df[(df["region_txt"]=='East Asia') | (df["region_txt"]=='Southeast Asia')]
df_asia.loc[df_asia['country_txt'] == 'Hong Kong', 'country_txt'] = gdf.loc[197]['country']
df_asia.loc[df_asia['country_txt'] == 'Macau', 'country_txt'] = gdf.loc[196]['country']
df_asia.loc[df_asia['country_txt'] == 'South Vietnam', 'country_txt'] = gdf.loc[3]['country']
df_asia.head(3)

C:\Users\Adam\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,iyear,imonth,iday,country,country_txt,region_txt,provstate,city,latitude,longitude,...,attacktype1_txt,targtype1,targtype1_txt,targsubtype1,targsubtype1_txt,natlty1,natlty1_txt,gname,weaptype1,weaptype1_txt
2,1970,1,0,160,Philippines,Southeast Asia,Tarlac,Unknown,15.478598,120.599741,...,Assassination,10,Journalists & Media,54.0,Radio Journalist/Staff/Facility,217.0,United States,Unknown,13,Unknown
4,1970,1,0,101,Japan,East Asia,Fukouka,Fukouka,33.580412,130.396361,...,Facility/Infrastructure Attack,7,Government (Diplomatic),46.0,Embassy/Consulate,217.0,United States,Unknown,8,Incendiary
26,1970,1,21,160,Philippines,Southeast Asia,Metropolitan Manila,Quezon City,14.674280,121.057495,...,Bombing/Explosion,7,Government (Diplomatic),46.0,Embassy/Consulate,217.0,United States,Unknown,6,Explosives


In [184]:
countries = df_asia.pivot_table(index=['country_txt'], aggfunc='size')

In [186]:
pd.set_option('display.max_rows', None)
countries_asia = countries.to_frame()
type(countries_asia)
countries_asia.reset_index(inplace=True)
countries_asia.columns = ['country', 'attacks']
countries_asia.sort_values('country')
countries_asia.sort_values('attacks')

,country,attacks
11,North Korea,1
0,Brunei,6
13,Singapore,7
3,East Timor,10
17,Vietnam,13
4,Hong Kong S.A.R.,26
7,Laos,27
8,Macao S.A.R,33
14,South Korea,38
15,Taiwan,50


In [187]:
countries_asia.max(axis=0)

country    Vietnam
attacks       6908
dtype: object

In [188]:
gdf_asia = gdf[gdf.country == countries_asia.loc[0]['country']]
for x in range(1, len(countries_asia['country'])):
    help_1 = gdf[gdf.country == countries_asia.loc[x]['country']]
    gdf_asia = pd.concat([gdf_asia, help_1], ignore_index=True)
gdf_asia

,country,country_code,geometry
0,Brunei,BRN,"MULTIPOLYGON (((115.14004 4.89976, 115.16846 4..."
1,Cambodia,KHM,"MULTIPOLYGON (((103.04512 11.28506, 103.02734 ..."
2,China,CHN,"MULTIPOLYGON (((118.18301 24.49629, 118.14951 ..."
3,East Timor,TLS,"MULTIPOLYGON (((125.64609 -8.13994, 125.57949 ..."
4,Hong Kong S.A.R.,HKG,"MULTIPOLYGON (((114.01543 22.51191, 114.01826 ..."
5,Indonesia,IDN,"MULTIPOLYGON (((97.48154 1.46509, 97.69834 1.1..."
6,Japan,JPN,"MULTIPOLYGON (((133.37051 36.20386, 133.32471 ..."
7,Laos,LAO,"POLYGON ((102.12744 22.37920, 102.18301 22.284..."
8,Macao S.A.R,MAC,"POLYGON ((113.47891 22.19556, 113.48105 22.217..."
9,Malaysia,MYS,"MULTIPOLYGON (((100.11914 6.44199, 100.13799 6..."


In [88]:
#Perform left merge to preserve every row in gdf.
merged = gdf_asia.merge(countries_asia, left_on = 'country', right_on = 'country', how = 'left')
#Replace NaN values to string 'No data'.
merged.fillna('No data', inplace = True)

In [89]:
import json
#Read data to json.
merged_json = json.loads(merged.to_json())
#Convert to String like object.
json_data = json.dumps(merged_json)

In [112]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure, show
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar, LogColorMapper, LogTicker, BasicTicker, ContinuousColorMapper, LinearAxis, LogTicker, FixedTicker, FuncTickFormatter, Legend, LegendItem, SingleIntervalTicker
from bokeh.palettes import brewer
import numpy as np


#Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource(geojson = json_data)

palette = brewer['YlGnBu'][8]

#Reverse color order so that dark blue is highest obesity.
palette = palette[::-1]

#Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
#color_mapper = LogColorMapper(palette = palette, low=1, high=1e4)


#Create figure object.
p = figure(title = 'Terrorism attacks, 1970 - 2018', plot_height = 700 , plot_width = 800, toolbar_location = None)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None



base_colors = palette #['#01DF01', '#74DF00','#A5DF00', '#D7DF01','#dba901','#DF7401','#DF3A01' ,'#DF0101']
bounds = [0, 10, 20, 50, 100, 500, 1000, 5000, 7000]
low = 0
high = 7000


# these are a dummy glyphs to help draw the legend
dummy_for_legend = [p.line(x=[100, 100], y=[0, 0], line_width=15, color=c, name='dummy_for_legend')
                    for c in base_colors]
legend_labels = [f'     < {bounds[1]}'] + \
                [('' if l < 0 else '     ' if l < 10 else '   ' if l < 100 else ' ')
                 + f'{l} ‒ {h}' for l, h in zip(bounds[1:], bounds[2:-1])] + \
                [f'     > {bounds[-2]}']

legend1 = Legend(title="Legend", background_fill_color='white',
                 items=[LegendItem(label=lab, renderers=[gly]) for lab, gly in zip(legend_labels, dummy_for_legend) ])



bound_colors = []
j = 0
for i in range(low, high, 5):
    if i >= bounds[j+1]:
        j += 1
    bound_colors.append(base_colors[j])
color_mapper = LinearColorMapper(palette=bound_colors, low=low, high=high, nan_color="white")




#Add patch renderer to figure. 
p.patches('xs','ys', source = geosource,fill_color = {'field' :'attacks', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.125, fill_alpha = 1)

p.add_layout(legend1)
p.legend.location = "bottom_left"
#Display figure inline in Jupyter Notebook.
output_notebook()
#Display figure.
show(p)

Loading BokehJS ...